## Homework 02: Unsupervised embedding-based MT
*Note: this homework is based on open materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [228]:
import gensim
import numpy as np
from gensim.models import KeyedVectors
import nltk
from sklearn.linear_model import LinearRegression
from collections import defaultdict
from nltk.probability import FreqDist


import gzip
from IPython.display import clear_output
import pdb

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [ ]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz -nc
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz -nc
clear_output()

In [14]:
uk_emb = KeyedVectors.load_word2vec_format('cc.uk.300.vec.gz')

In [16]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec.gz")

In [17]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153915405273),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222575426101685),
 ('октябрь', 0.9095538854598999),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796275138855),
 ('февраль', 0.8401416540145874)]

In [18]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999999403953552),
 ('липень', 0.9096440076828003),
 ('вересень', 0.901697039604187),
 ('червень', 0.8992519378662109),
 ('жовтень', 0.8810408711433411),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431818008423)]

In [19]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829231202602386),
 ('Ведущий-Алексей', 0.2443869560956955),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981399834156036),
 ('лексей', 0.23740756511688232),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222028017044)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [20]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [24]:
!wget -O ukr_rus.train.txt http://tiny.cc/jfgecz
clear_output()

In [25]:
!wget -O ukr_rus.test.txt http://tiny.cc/6zoeez
clear_output()

In [26]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [27]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [32]:
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [33]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531403541564941),
 ('июнь', 0.8402307033538818),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.8331867456436157),
 ('февраль', 0.8311493396759033),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244150280952454),
 ('июль', 0.8228995203971863),
 ('август', 0.8112362027168274),
 ('январь', 0.8022859692573547)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [110]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        top_pred = ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn)
        for j in range(topn):
            if top_pred[j][0] == ru:
                num_matches = num_matches + 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [111]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [115]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [113]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [114]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [119]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, _, V_T = np.linalg.svd(X_train.T @ Y_train)
    return U @ V_T

In [120]:
W = learn_transform(X_train, Y_train)

In [121]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.805662989616394),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032935261726379),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923861145973206),
 ('декабрь', 0.7715375423431396)]

In [122]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [199]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
clear_output()

In [124]:
!gzip -d ./uk.txt.gz

In [125]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

In [126]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [ ]:
nltk.download('punkt')
uk_corpus = [' '.join(nltk.word_tokenize(x.lower())) for x in uk_corpus]

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

One possible method besides adding UNK token, was that I just iterated over the Russian sentenses and pulled out words that come after some given token. Then I picked the most frequently occuring word out of the ones I got. Hence, statistically this will give the highest chance of correct translation.

In [230]:
# import russian text

!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/ru.txt.gz
!gzip -d ./ru.txt.gz
with open('./ru.txt', 'r') as f:
    ru_corpus = f.readlines()
clear_output()

In [231]:
next_word_dict = defaultdict(list)

# iterate over text and get list of words occuring after token
for sentence in ru_corpus:
    sentence_list = nltk.word_tokenize(sentence.lower())
    for i in range(len(sentence_list)):
        next_word_dict[sentence_list[i]].append(sentence_list[(i + 1) % len(sentence_list)])

# get the most frequently occuring consecutive token
for k, v in next_word_dict.items():  
    fdist = FreqDist(v)
    next_word_dict[k] = fdist.most_common(1)[0][0]

In [235]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    original = nltk.word_tokenize(sentence)
    translated = []
    for word in original:
        if word in uk_emb:
            translated.append((ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0]))
            temp = ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0]
        else:
            #translated.append('UNK')
            translated.append(next_word_dict[temp])

    return " ".join(translated)

In [236]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [237]:
for sent in uk_corpus[:20]:
    print(translate(sent))

мной уже закончу колледж , когда мы прибежишь со америки .
он велел мне немедленно выйти со комнаты .
как бы ты не пытался , ты не выучишь английский за два-три месяца .
пока мной не позвонил , он не пришел .
во вселенной много галактик .
она принимает души утрам .
непослушный мальчик заблудился и проглядывался по сторонам .
она медленно исчезала во туманном лесу .
наш самолет летел свыше облаками .
во майка То несколько друзей во сколько .
город бомбили враждебные самолеты .
мной встретиться со тобой во субботу 00 третий .
для финансирование войны было издан облигации .
откуда принимают начало олимпийские игры ?
мной сказал себе : « это красивая идея » .
мы собирались пробыть там возле двух недель .
как по меня , то сейчас промолчу .
мой дядя вчера умер от рака желудке .
кажется , дети устали от плавание .
нет любовь без ревности .
